# Twitter Bot - Post, Read, and Reply

This notebook demonstrates how to create a Twitter bot that can:
- Post tweets
- Read tweets from timeline
- Reply to tweets

Using Twitter API v2 with OAuth 2.0 authentication.

## 1. Import Dependencies



In [5]:
from dotenv import load_dotenv
from datetime import datetime
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Twitter API Credentials
# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
API_KEY = os.getenv('API_KEY')
API_KEY_SECRET = os.getenv('API_KEY_SECRET')
BEARER_TOKEN = os.getenv('BEARER_TOKEN')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

# Verify that all credentials are loaded
required_vars = [API_KEY, API_KEY_SECRET, BEARER_TOKEN, ACCESS_TOKEN, ACCESS_TOKEN_SECRET]
missing_vars = [var for var in required_vars if not var]

if missing_vars:
    raise ValueError(f"Missing required environment variables: {missing_vars}")

print("✅ All Twitter API credentials loaded successfully from .env file!")

✅ All Twitter API credentials loaded successfully from .env file!


## 2. Initialize Twitter API Client

We'll use both OAuth 1.0a (for v1.1 API) and OAuth 2.0 (for v2 API) to access different endpoints.

In [3]:
import tweepy
class TwitterBot:
    def __init__(self):
        # OAuth 1.0a authentication for API v1.1 and v2
        auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        
        # API v1.1 client (for some legacy endpoints)
        self.api_v1 = tweepy.API(auth, wait_on_rate_limit=True)
        
        # API v2 client (recommended for new applications)
        self.client = tweepy.Client(
            bearer_token=BEARER_TOKEN,
            consumer_key=API_KEY,
            consumer_secret=API_KEY_SECRET,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET,
            wait_on_rate_limit=True
        )
        
        logger.info("Twitter Bot initialized successfully!")
    
    def verify_credentials(self):
        """Verify that the API credentials are working"""
        try:
            user = self.client.get_me()
            logger.info(f"Authenticated as: @{user.data.username}")
            return True
        except Exception as e:
            logger.error(f"Authentication failed: {e}")
            return False

# Initialize the bot
bot = TwitterBot()

# Verify credentials
if bot.verify_credentials():
    print("✅ Twitter Bot is ready to use!")
else:
    print("❌ Failed to authenticate. Please check your credentials.")

2025-09-06 19:29:09,409 - INFO - Twitter Bot initialized successfully!
2025-09-06 19:29:10,156 - INFO - Authenticated as: @RaviDevgam


✅ Twitter Bot is ready to use!


## 3. Post a Tweet

Function to post a new tweet.

In [6]:
def post_tweet(text):
    """Post a tweet with the given text"""
    try:
        # Check tweet length (280 characters max)
        if len(text) > 280:
            logger.warning(f"Tweet too long ({len(text)} characters). Truncating...")
            text = text[:277] + "..."
        
        response = bot.client.create_tweet(text=text)
        tweet_id = response.data['id']
        logger.info(f"Tweet posted successfully! Tweet ID: {tweet_id}")
        return tweet_id
    except Exception as e:
        logger.error(f"Failed to post tweet: {e}")
        return None

# Example usage
sample_tweet = f"Hello Twitter! 🤖 This is my automated bot posting at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} #TwitterBot #Python"
tweet_id = post_tweet(sample_tweet)

if tweet_id:
    print(f"✅ Tweet posted! ID: {tweet_id}")
else:
    print("❌ Failed to post tweet")

2025-09-06 19:30:23,320 - INFO - Tweet posted successfully! Tweet ID: 1964327825252487259


✅ Tweet posted! ID: 1964327825252487259


## 4. Read Tweets from Timeline

Function to read tweets from your home timeline.

In [7]:
def read_home_timeline(count=4):
    """Read tweets from home timeline"""
    try:
        # Get home timeline using API v1.1 (more reliable for timeline access)
        tweets = bot.api_v1.home_timeline(count=count, tweet_mode='extended')
        
        timeline_data = []
        for tweet in tweets:
            tweet_data = {
                'id': tweet.id,
                'user': tweet.user.screen_name,
                'text': tweet.full_text,
                'created_at': tweet.created_at,
                'retweet_count': tweet.retweet_count,
                'favorite_count': tweet.favorite_count
            }
            timeline_data.append(tweet_data)
        
        logger.info(f"Retrieved {len(timeline_data)} tweets from timeline")
        return timeline_data
    except Exception as e:
        logger.error(f"Failed to read timeline: {e}")
        return []

# Read recent tweets
recent_tweets = read_home_timeline(5)

print(f"📱 Recent tweets from your timeline ({len(recent_tweets)} tweets):")
print("=" * 60)

for i, tweet in enumerate(recent_tweets, 1):
    print(f"{i}. @{tweet['user']} - {tweet['created_at']}")
    print(f"   {tweet['text'][:100]}{'...' if len(tweet['text']) > 100 else ''}")
    print(f"   ❤️ {tweet['favorite_count']} | 🔄 {tweet['retweet_count']}")
    print("-" * 60)

2025-09-06 19:31:44,959 - ERROR - Failed to read timeline: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product


📱 Recent tweets from your timeline (0 tweets):


## 5. Search for Tweets

Function to search for tweets by keyword or hashtag.

In [ ]:
def search_tweets(query, count=3):
    """Search for tweets using a query"""
    try:
        # Use API v2 for better search capabilities
        tweets = tweepy.Paginator(
            bot.client.search_recent_tweets,
            query=query,
            tweet_fields=['created_at', 'author_id', 'public_metrics'],
            user_fields=['username'],
            expansions=['author_id'],
            max_results=min(count, 100)  # API limit
        ).flatten(limit=count)
        
        search_results = []
        for tweet in tweets:
            tweet_data = {
                'id': tweet.id,
                'text': tweet.text,
                'created_at': tweet.created_at,
                'author_id': tweet.author_id,
                'public_metrics': tweet.public_metrics
            }
            search_results.append(tweet_data)
        
        logger.info(f"Found {len(search_results)} tweets for query: {query}")
        return search_results
    except Exception as e:
        logger.error(f"Failed to search tweets: {e}")
        return []

# Example search
search_query = "#Python -is:retweet"  # Search for Python tweets, exclude retweets
found_tweets = search_tweets(search_query, 3)

print(f"🔍 Search results for '{search_query}' ({len(found_tweets)} tweets):")
print("=" * 60)

for i, tweet in enumerate(found_tweets, 1):
    print(f"{i}. Tweet ID: {tweet['id']}")
    print(f"   {tweet['text'][:150]}{'...' if len(tweet['text']) > 150 else ''}")
    print(f"   📅 {tweet['created_at']}")
    if 'public_metrics' in tweet:
        metrics = tweet['public_metrics']
        print(f"   ❤️ {metrics.get('like_count', 0)} | 🔄 {metrics.get('retweet_count', 0)}")
    print("-" * 60)

## 6. Reply to a Tweet

Function to reply to a specific tweet.

In [ ]:
def reply_to_tweet(tweet_id, reply_text):
    """Reply to a specific tweet"""
    try:
        # Check reply length
        if len(reply_text) > 280:
            logger.warning(f"Reply too long ({len(reply_text)} characters). Truncating...")
            reply_text = reply_text[:277] + "..."
        
        response = bot.client.create_tweet(
            text=reply_text,
            in_reply_to_tweet_id=tweet_id
        )
        
        reply_id = response.data['id']
        logger.info(f"Reply posted successfully! Reply ID: {reply_id}")
        return reply_id
    except Exception as e:
        logger.error(f"Failed to post reply: {e}")
        return None

# Example: Reply to the first tweet from search results (if any)
if found_tweets:
    target_tweet_id = found_tweets[0]['id']
    reply_message = "Great post about Python! 🐍 Keep up the awesome work! #PythonCommunity"
    
    print(f"\n💬 Attempting to reply to tweet ID: {target_tweet_id}")
    reply_id = reply_to_tweet(target_tweet_id, reply_message)
    
    if reply_id:
        print(f"✅ Reply posted! Reply ID: {reply_id}")
    else:
        print("❌ Failed to post reply")
else:
    print("No tweets found to reply to. Try running the search cell first.")

## 7. Advanced Bot Functions

Additional useful functions for your Twitter bot.

In [ ]:
def like_tweet(tweet_id):
    """Like a specific tweet"""
    try:
        bot.client.like(tweet_id)
        logger.info(f"Liked tweet: {tweet_id}")
        return True
    except Exception as e:
        logger.error(f"Failed to like tweet: {e}")
        return False

def retweet(tweet_id):
    """Retweet a specific tweet"""
    try:
        bot.client.retweet(tweet_id)
        logger.info(f"Retweeted: {tweet_id}")
        return True
    except Exception as e:
        logger.error(f"Failed to retweet: {e}")
        return False

def get_my_tweets(count=10):
    """Get your own recent tweets"""
    try:
        # Get authenticated user's ID
        user = bot.client.get_me()
        user_id = user.data.id
        
        tweets = bot.client.get_users_tweets(
            id=user_id,
            max_results=min(count, 100),
            tweet_fields=['created_at', 'public_metrics']
        )
        
        my_tweets = []
        if tweets.data:
            for tweet in tweets.data:
                tweet_data = {
                    'id': tweet.id,
                    'text': tweet.text,
                    'created_at': tweet.created_at,
                    'public_metrics': tweet.public_metrics
                }
                my_tweets.append(tweet_data)
        
        logger.info(f"Retrieved {len(my_tweets)} of your tweets")
        return my_tweets
    except Exception as e:
        logger.error(f"Failed to get your tweets: {e}")
        return []

# Example usage
my_recent_tweets = get_my_tweets(3)

print(f"\n📝 Your recent tweets ({len(my_recent_tweets)} tweets):")
print("=" * 60)

for i, tweet in enumerate(my_recent_tweets, 1):
    print(f"{i}. Tweet ID: {tweet['id']}")
    print(f"   {tweet['text'][:100]}{'...' if len(tweet['text']) > 100 else ''}")
    print(f"   📅 {tweet['created_at']}")
    if 'public_metrics' in tweet:
        metrics = tweet['public_metrics']
        print(f"   ❤️ {metrics.get('like_count', 0)} | 🔄 {metrics.get('retweet_count', 0)}")
    print("-" * 60)

## 8. Automated Bot Workflow

A simple automated workflow that combines all the functions.

In [ ]:
def automated_bot_workflow():
    """Automated workflow: search, like, and sometimes reply"""
    try:
        # Search for interesting tweets
        search_queries = [
            "#Python programming -is:retweet",
            "#MachineLearning -is:retweet",
            "#DataScience -is:retweet"
        ]
        
        for query in search_queries:
            print(f"\n🔍 Searching for: {query}")
            tweets = search_tweets(query, 2)  # Get 2 tweets per query
            
            for tweet in tweets:
                tweet_id = tweet['id']
                
                # Like the tweet
                if like_tweet(tweet_id):
                    print(f"   ❤️ Liked tweet: {tweet_id}")
                
                # Randomly decide to reply (30% chance)
                import random
                if random.random() < 0.3:
                    replies = [
                        "Interesting post! Thanks for sharing! 👍",
                        "Great insights! 🤖",
                        "Love this! Keep up the great work! 🚀"
                    ]
                    reply_text = random.choice(replies)
                    
                    if reply_to_tweet(tweet_id, reply_text):
                        print(f"   💬 Replied to tweet: {tweet_id}")
                
                # Be respectful - add delay between actions
                time.sleep(2)
        
        print("\n✅ Automated workflow completed!")
        
    except Exception as e:
        logger.error(f"Error in automated workflow: {e}")

# Uncomment the line below to run the automated workflow
# automated_bot_workflow()

print("🤖 Bot functions are ready!")
print("\nAvailable functions:")
print("- post_tweet(text): Post a new tweet")
print("- read_home_timeline(count): Read tweets from timeline")
print("- search_tweets(query, count): Search for tweets")
print("- reply_to_tweet(tweet_id, text): Reply to a tweet")
print("- like_tweet(tweet_id): Like a tweet")
print("- retweet(tweet_id): Retweet a tweet")
print("- get_my_tweets(count): Get your recent tweets")
print("- automated_bot_workflow(): Run automated workflow")

## 9. Bot State Management

Save and load bot state to avoid repeating actions.

In [ ]:
import json
from datetime import datetime, timedelta

class BotState:
    def __init__(self, state_file='bot_state.json'):
        self.state_file = state_file
        self.state = self.load_state()
    
    def load_state(self):
        """Load bot state from file"""
        try:
            with open(self.state_file, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            return {
                'processed_tweets': [],
                'last_timeline_check': None,
                'daily_post_count': 0,
                'last_post_date': None
            }
    
    def save_state(self):
        """Save current state to file"""
        try:
            with open(self.state_file, 'w') as f:
                json.dump(self.state, f, indent=2, default=str)
        except Exception as e:
            logger.error(f"Failed to save state: {e}")
    
    def is_tweet_processed(self, tweet_id):
        """Check if a tweet has already been processed"""
        return str(tweet_id) in self.state.get('processed_tweets', [])
    
    def mark_tweet_processed(self, tweet_id):
        """Mark a tweet as processed"""
        if 'processed_tweets' not in self.state:
            self.state['processed_tweets'] = []
        
        self.state['processed_tweets'].append(str(tweet_id))
        
        # Keep only last 1000 processed tweets to prevent file from growing too large
        if len(self.state['processed_tweets']) > 1000:
            self.state['processed_tweets'] = self.state['processed_tweets'][-1000:]
        
        self.save_state()
    
    def can_post_today(self, max_posts_per_day=10):
        """Check if bot can post more tweets today"""
        today = datetime.now().date().isoformat()
        last_post_date = self.state.get('last_post_date')
        
        if last_post_date != today:
            # Reset daily counter for new day
            self.state['daily_post_count'] = 0
            self.state['last_post_date'] = today
            self.save_state()
        
        return self.state.get('daily_post_count', 0) < max_posts_per_day
    
    def increment_daily_posts(self):
        """Increment daily post counter"""
        today = datetime.now().date().isoformat()
        self.state['last_post_date'] = today
        self.state['daily_post_count'] = self.state.get('daily_post_count', 0) + 1
        self.save_state()

# Initialize bot state
bot_state = BotState()

print("💾 Bot state management initialized!")
print(f"📊 Current state: {bot_state.state}")

## 10. Safe Posting Function with Rate Limiting

Enhanced posting function that respects rate limits and daily posting limits.

In [ ]:
def safe_post_tweet(text, max_daily_posts=10):
    """Safely post a tweet with rate limiting and daily limits"""
    try:
        # Check daily posting limit
        if not bot_state.can_post_today(max_daily_posts):
            logger.warning(f"Daily posting limit reached ({max_daily_posts} posts)")
            return None
        
        # Post the tweet
        tweet_id = post_tweet(text)
        
        if tweet_id:
            # Update daily post count
            bot_state.increment_daily_posts()
            logger.info(f"Daily posts: {bot_state.state['daily_post_count']}/{max_daily_posts}")
        
        return tweet_id
    except Exception as e:
        logger.error(f"Error in safe_post_tweet: {e}")
        return None

# Example usage
test_message = f"Testing safe posting function! 🧪 Current time: {datetime.now().strftime('%H:%M:%S')}"
result = safe_post_tweet(test_message)

if result:
    print(f"✅ Safely posted tweet: {result}")
else:
    print("❌ Could not post tweet (may have hit daily limit)")

## 🎉 Congratulations!

Your Twitter bot is now ready! Here's what you can do:

### ✅ **Bot Capabilities:**
- ✅ Post tweets with automatic length checking
- ✅ Read your home timeline
- ✅ Search for tweets by keywords/hashtags
- ✅ Reply to specific tweets
- ✅ Like and retweet posts
- ✅ Get your own recent tweets
- ✅ Automated workflows
- ✅ State management to avoid duplicate actions
- ✅ Daily posting limits for responsible usage

### 🔒 **Security Best Practices:**
- Move credentials to environment variables in production
- Use `.env` file for local development
- Never commit credentials to version control
- Monitor your bot's activity regularly

### 🚀 **Next Steps:**
1. Test each function individually
2. Customize the automated workflow
3. Add more sophisticated content generation
4. Implement sentiment analysis for smarter replies
5. Add scheduling capabilities

**Happy Tweeting! 🐦**